In [1]:
import pandas as pd
import os

In [2]:
os.chdir("../../")

In [3]:
os.getcwd()

'd:\\DeepFakeClassification'

In [6]:
df=pd.read_csv("archive/KAGGLE/DATASET-balanced.csv",sep=',')

In [9]:
df.shape

(11778, 27)

In [15]:
len(os.listdir('./archive/KAGGLE/AUDIO/FAKE'))

56

In [16]:
len(os.listdir('./archive/KAGGLE/AUDIO/REAL'))

8

In [24]:
import os
import wave

In [28]:
def find_duration(directory):
    total_duration = 0
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)

        if file_path.endswith(".wav") and os.path.isfile(file_path):  # Ensure it's a WAV file
            try:
                with wave.open(file_path, 'r') as audio_file:
                    frames = audio_file.getnframes()
                    rate = audio_file.getframerate()
                    duration = frames / float(rate)
                    total_duration += duration
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

    print(f"Total duration of all WAV files: {total_duration/3600} hours")

In [29]:
find_duration('./archive/KAGGLE/AUDIO/FAKE')


Total duration of all WAV files: 7.279096031746032 hours


In [ ]:
find_duration('./archive/KAGGLE/AUDIO/REAL')

Total duration of all WAV files: 1.0402023446948225 hours


In [32]:
import os
import random
import pandas as pd
from pydub import AudioSegment

e:\Python\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [53]:
input_dir = "./archive/KAGGLE/AUDIO/REAL"
output_dir = "./archive/KAGGLE/AUDIO/BROKEN_REAL"
manifest_path = os.path.join(output_dir, "manifest.csv")

In [54]:
# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Initialize manifest data
manifest_data = []

# Process each WAV file
for file_name in os.listdir(input_dir):
    if file_name.endswith(".wav"):
        file_path = os.path.join(input_dir, file_name)
        audio = AudioSegment.from_wav(file_path)
        duration = len(audio) / 1000  # Convert to seconds

        start_time = 0
        segment_count = 1

        while start_time + 4 <= duration:
            segment_length = random.randint(4, 10)  # Random segment length (4-10 sec)
            end_time = min(start_time + segment_length, duration)

            # Ensure the last segment is at least 4 sec
            if duration - end_time < 4:
                break

            # Extract segment
            segment = audio[start_time * 1000:end_time * 1000]

            # Save new segment
            segment_file_name = f"{os.path.splitext(file_name)[0]}_part{segment_count}.wav"
            segment_output_path = os.path.join(output_dir, segment_file_name)
            segment.export(segment_output_path, format="wav")

            # Store absolute path and duration
            absolute_path = os.path.abspath(segment_output_path)
            segment_duration = end_time - start_time  # Duration of this segment
            manifest_data.append([absolute_path, segment_duration])

            # Move to next segment
            start_time = end_time
            segment_count += 1

# Save manifest (with absolute file paths and durations)
df = pd.DataFrame(manifest_data, columns=["audio_path", "duration"])
df.to_csv(manifest_path, index=False)

print("Processing complete! Manifest saved at:", manifest_path)

Processing complete! Manifest saved at: ./archive/KAGGLE/AUDIO/BROKEN_REAL\manifest.csv


In [55]:
fake=pd.read_csv('./archive/KAGGLE/AUDIO/BROKEN_FAKE/manifest.csv')
real=pd.read_csv('./archive/KAGGLE/AUDIO/BROKEN_REAL/manifest.csv')

In [56]:
import random

In [57]:
# Function to sample 10 minutes worth of audio files
def sample_10_minutes(df):
    sampled_files = []
    total_duration = 0
    
    # Shuffle dataframe to ensure randomness
    df = df.sample(frac=1).reset_index(drop=True)

    for _, row in df.iterrows():
        if total_duration + row['duration'] > 600:  # 10 min = 600 sec
            continue
        sampled_files.append(row)
        total_duration += row['duration']
        
        # Stop if 10 min is reached
        if total_duration >= 600:
            break

    return pd.DataFrame(sampled_files)

In [58]:
# Sample from fake and real datasets
fake_sampled = sample_10_minutes(fake)
real_sampled = sample_10_minutes(real)

In [59]:
# Add labels (1 for fake, 0 for real)
fake_sampled['label'] = 1
real_sampled['label'] = 0

In [60]:
final_df = pd.concat([fake_sampled, real_sampled]).sample(frac=1).reset_index(drop=True)
final_df.to_csv('./archive/KAGGLE/AUDIO/COMBINED_MANIFEST.csv', index=False)